In [ ]:
#import modules
import numpy as np
from pyspark.sql.types import *
from pyspark.sql.functions import *

# start
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark import StorageLevel

# sc = SparkContext(master="local[4]")
appName = "MapReduce in Spark"
spark = SparkSession.builder.master("local[*]") \
        .appName(appName) \
        .config("spark.driver.memory", "60g") \
        .config('spark.executor.memory','60g') \
        .getOrCreate()
spark

In [3]:
#define our schema
flightSchema = StructType([
  StructField("Time", FloatType()),
  StructField("V1", FloatType()),
  StructField("V2", FloatType()),
  StructField("V3", FloatType()),
  StructField("V4", FloatType()),
  StructField("V5", FloatType()),
  StructField("V6", FloatType()),
  StructField("V7", FloatType()),
  StructField("V8", FloatType()),
  StructField("V9", FloatType()),
  StructField("V10", FloatType()),
  StructField("V11", FloatType()),
  StructField("V12", FloatType()),
  StructField("V13", FloatType()),
  StructField("V14", FloatType()),
  StructField("V15", FloatType()),
  StructField("V16", FloatType()),
  StructField("V17", FloatType()),
  StructField("V18", FloatType()),
  StructField("V19", FloatType()),
  StructField("V20", FloatType()),
  StructField("V21", FloatType()),
  StructField("V22", FloatType()),
  StructField("V23", FloatType()),
  StructField("V24", FloatType()),
  StructField("V25", FloatType()),
  StructField("V26", FloatType()),
  StructField("V27", FloatType()),
  StructField("V28", FloatType()),
  StructField("Amount", FloatType()),
  StructField("Class", FloatType()),
])

csv = spark.read.csv('C:/Spark/creditcard.csv',schema=flightSchema, header=True)

In [4]:
csv.groupby("Class").count().show()

+-----+------+
|Class| count|
+-----+------+
|  1.0|   492|
|  0.0|284315|
+-----+------+



In [5]:
csv_final = csv.drop('Time','Amount')
csv_final

DataFrame[V1: float, V2: float, V3: float, V4: float, V5: float, V6: float, V7: float, V8: float, V9: float, V10: float, V11: float, V12: float, V13: float, V14: float, V15: float, V16: float, V17: float, V18: float, V19: float, V20: float, V21: float, V22: float, V23: float, V24: float, V25: float, V26: float, V27: float, V28: float, Class: float]

In [6]:
dividedData2 = csv_final.randomSplit([0.8, 0.2])
trainingData2 = dividedData2[0] #index 0 = data training
testingData2 = dividedData2[1] #index 1 = data testing
train_rows2 = trainingData2.count()
test_rows2 = testingData2.count()
print ("Training data rows:", train_rows2, "; Testing data rows:", test_rows2)

Training data rows: 227300 ; Testing data rows: 57507


In [7]:
trainlist = []
for i in trainingData2.collect():
    trainlist.append(tuple(i))

In [8]:
testlist = []
for j in testingData2.collect():
    testlist.append(tuple(j))

In [26]:
import math
import time
mydict = {}
result = []
predict_class = []
columns = ["key","values"]
result_predict = []
start_time = time.time()
k = 3
i = 0
for row in testingData2.take(1):
    for row2 in trainingData2.collect():
        x = math.sqrt(np.square((np.array(row[:-1]) - np.array(row2[:-1]))).sum())
        
        upd = {x:row2[-1]}
        mydict.update(upd)  
        
#         result.append(x)
#         predict_class.append(row2[-1])  
        
    data = list(zip(result,predict_class))
    df = spark.createDataFrame(mydict,columns).sort("key").take(3)
    total = df[0][1]+ df[1][1] + df[2][1]
    if total <= 1.5:
        predict = 0.0
    else:
        predict = 1.0
        
    result_predict.append(predict)
    result = []
    data =[]
    predict_class = []
    i += 1
    print(i,total)
print("finish in ",time.time() - start_time)

TypeError: Can not infer schema for type: <class 'float'>

In [23]:
dataDictionary = [
        ('James',{'hair':'black','eye':'brown'}),
        ('Michael',{'hair':'brown','eye':None}),
        ('Robert',{'hair':'red','eye':'black'}),
        ('Washington',{'hair':'red','eye':'grey'}),
        ('Jefferson',{'hair':'red','eye':''})
        ]

In [24]:
df = spark.createDataFrame(data=dataDictionary, schema = ["name","properties"])
df.printSchema()
df.show(truncate=False)

root
 |-- name: string (nullable = true)
 |-- properties: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)

+----------+-----------------------------+
|name      |properties                   |
+----------+-----------------------------+
|James     |[eye -> brown, hair -> black]|
|Michael   |[eye ->, hair -> brown]      |
|Robert    |[eye -> black, hair -> red]  |
|Washington|[eye -> grey, hair -> red]   |
|Jefferson |[eye -> , hair -> red]       |
+----------+-----------------------------+



In [27]:
mydict

{3.620758157265176: 1.0,
 2.4138383190960706: 1.0,
 1.2069182438380497: 1.0,
 85.56165172440828: 0.0,
 1.2069183762147764: 1.0,
 91.97054841757115: 0.0,
 76.16204913542397: 0.0,
 3.714798912001451: 0.0,
 4.756762916611971: 1.0,
 5.962164341554695: 1.0,
 55.61828870336105: 0.0,
 7.168076754643554: 1.0,
 56.39591127355932: 0.0,
 8.374275958548424: 1.0,
 84.25701725276393: 0.0,
 9.580655482769911: 1.0,
 71.54290755435223: 0.0,
 10.787153673184317: 1.0,
 55.24079263628228: 0.0,
 11.99373398807467: 1.0,
 57.48965816225156: 0.0,
 60.28311919551122: 0.0,
 55.02157676468051: 0.0,
 14.407055556276047: 1.0,
 55.047466063432516: 0.0,
 15.613770704147154: 1.0,
 55.32012587529035: 0.0,
 16.820511974395476: 1.0,
 18.03707787946245: 1.0,
 55.83974318691567: 0.0,
 76.47816753461811: 0.0,
 19.262464532389426: 0.0,
 56.48801362999178: 0.0,
 62.77138589749963: 0.0,
 28.823869241691746: 1.0,
 20.367779490847674: 1.0,
 56.939625549255055: 0.0,
 21.57429178148045: 1.0,
 30.78061754971559: 1.0,
 39.205349440